In [1]:
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
session = tf.Session(config=config)

# 设置session
KTF.set_session(session)

/home/wangxiaochen/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
#!/usr/bin/env
# coding:utf-8
"""
Created on 17/12/14 上午10:03

base Info
"""
__author__ = 'xiaochenwang94'
__version__ = '1.0'

import numpy as np
from keras import layers
from keras import regularizers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, \
    AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Merge, concatenate
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
# import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
from sklearn.model_selection import StratifiedKFold

import keras.backend as K

K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [3]:
# GRADED FUNCTION: identity_block

# GRADED FUNCTION: identity_block

def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. You'll need this later to add back to the main path.
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid',
               kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=regularizers.l2(0.01))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same',
               kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=regularizers.l2(0.01))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid',
               kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=regularizers.l2(0.01))(X)
    X = BatchNormalization(axis=3)(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    ### END CODE HERE ###

    return X

In [4]:
# GRADED FUNCTION: convolutional_block

# GRADED FUNCTION: convolutional_block

def convolutional_block(X, f, filters, stage, block, s=2):
    """
    Implementation of the convolutional block as defined in Figure 4

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used

    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path
    X = Conv2D(F1, (1, 1), strides=(s, s), kernel_initializer=glorot_uniform(seed=0),
               kernel_regularizer=regularizers.l2(0.01))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(F2, (f, f), strides=(1, 1), padding='same',
               kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=regularizers.l2(0.01))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(F3, (1, 1), strides=(1, 1), kernel_initializer=glorot_uniform(seed=0),
               kernel_regularizer=regularizers.l2(0.01))(X)
    X = BatchNormalization(axis=3)(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(F3, (1, 1), strides=(s, s),
                        kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=regularizers.l2(0.01))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    ### END CODE HERE ###

    return X

In [5]:
# GRADED FUNCTION: ResNet50

def ResNet50(input_shape=(64, 64, 3), layer_num=11):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    classes = 1
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(1, 1), kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    if layer_num >= 15:
        # Stage 3 (≈4 lines)
        X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
        X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')

    if layer_num >= 20:
        # Stage 3 (≈4 lines)
        X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
        X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')

    # if layer_num >= 23:

    #         ### START CODE HERE ###

    #         # Stage 3 (≈4 lines)
    #         X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    #         X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    #         X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    #         X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    #     if layer_num >= 41:

    #         # Stage 4 (≈6 lines)
    #         X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    #         X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    #         X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    #         X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    #         X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    #         X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    #     if layer_num == 50:
    #         # Stage 5 (≈3 lines)
    #         X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    #         X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    #         X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    #     # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    #     X = AveragePooling2D((2, 2), name='avg_pool')(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    #     X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    # X = Dense(1, name='fc' + str(classes), kernel_initializer=glorot_uniform(seed=0))(X)

    return X, X_input

In [6]:
def Three_ResNet(R1, R2, R3, X_inputs):

    X = concatenate([R1, R2, R3])
    X = Dense(1, name='fc', kernel_initializer=glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs=X_inputs, outputs=X, name='ThreeResNet50')

    return model

In [7]:
from sklearn.cross_validation import train_test_split
stores = np.load('./ResNetData/stores.npy')
print(stores.shape)
stores = stores.reshape(stores.shape[0], 9, 9,-1)
pois = np.load('./ResNetData/pois.npy')
pois = pois.reshape(pois.shape[0],9,9,-1)
roads = np.load('./ResNetData/roads.npy')
roads = roads.reshape(roads.shape[0],9,9,-1)

y_data = np.load('./ResNetData/shopPower_y.npy')
origin_y = y_data.copy()
# data = data.reshape((24331, 9, 9, 52))
y_data = y_data/10

/home/wangxiaochen/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(24331, 81, 33)


In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(stores.reshape(stores.shape[0], -1))
stores = scaler.transform(stores.reshape(stores.shape[0], -1))
stores = stores.reshape(stores.shape[0], 9, 9, -1)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(pois.reshape(pois.shape[0], -1))
pois = scaler.transform(pois.reshape(pois.shape[0], -1))
pois = pois.reshape(pois.shape[0], 9, 9, -1)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(roads.reshape(roads.shape[0], -1))
roads = scaler.transform(roads.reshape(roads.shape[0], -1))
roads = roads.reshape(roads.shape[0], 9, 9, -1)

combine = np.concatenate((stores, pois, roads), axis=3)

In [9]:
def my_loss(y_true,y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred), axis=-1))

In [10]:
seed = 7
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []

In [41]:
mini_loss = 1000

In [42]:
model10_stores, instore = ResNet50(input_shape=stores.shape[1:])
model10_roads, inroad = ResNet50(input_shape=roads.shape[1:])
model10_pois, in3poi = ResNet50(input_shape=pois.shape[1:])

In [43]:
model10_combine = Three_ResNet(model10_stores, model10_roads, model10_pois, [instore, inroad, in3poi])
model10_combine.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [45]:
line = model10_combine.fit([stores[:1000], roads[:1000], pois[:1000]], y_data[:1000], epochs = 3, batch_size = 16, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/3
800/800 [==============================] - 9s 11ms/step - loss: 26.0270 - mean_squared_error: 2.8031 - val_loss: 23.8232 - val_mean_squared_error: 0.8254
Epoch 2/3
800/800 [==============================] - 9s 11ms/step - loss: 25.4388 - mean_squared_error: 2.6587 - val_loss: 25.8936 - val_mean_squared_error: 3.3439
Epoch 3/3
800/800 [==============================] - 9s 11ms/step - loss: 25.8626 - mean_squared_error: 3.5361 - val_loss: 26.1188 - val_mean_squared_error: 4.0244


In [46]:
line.history

{'loss': [26.027027168273925, 25.438829345703127, 25.862627105712889],
 'mean_squared_error': [2.8031145745515822,
  2.6586571151018141,
  3.5360728573799132],
 'val_loss': [23.82317840576172, 25.893581619262694, 26.11878646850586],
 'val_mean_squared_error': [0.82536059379577642,
  3.3438695907592773,
  4.0244360923767086]}

In [12]:
for train, test in kfold.split(stores, origin_y):
    model10_stores, instore = ResNet50(input_shape=stores.shape[1:])
    model10_roads, inroad = ResNet50(input_shape=roads.shape[1:])
    model10_pois, in3poi = ResNet50(input_shape=pois.shape[1:])
    
    model10_combine = Three_ResNet(model10_stores, model10_roads, model10_pois, [instore, inroad, in3poi])
    model10_combine.compile(optimizer='adam', loss='mse', metrics=['mse'])
    for _ in range(10):
        lines10_combine = model10_combine.fit([stores[train], roads[train], pois[train]], y_data[train], epochs = 5, batch_size = 16, validation_split=0.2)
        
        
    preds10_combine = model10_combine.evaluate([stores[test], roads[test], pois[test]], y_data[test])
    print ("model10_combine Loss = " + str(preds10_combine[0]))
    print ("model10_combine Test Accuracy = " + str(preds10_combine[1]))
    print(preds10_combine)
    cvscores.append(preds10_combine)

Train on 15569 samples, validate on 3893 samples
Epoch 1/5
15569/15569 [==============================] - 73s 5ms/step - loss: 26.6913 - mean_squared_error: 3.4850 - val_loss: 19.3628 - val_mean_squared_error: 0.6679
Epoch 2/5
15569/15569 [==============================] - 64s 4ms/step - loss: 14.6850 - mean_squared_error: 0.6838 - val_loss: 11.9365 - val_mean_squared_error: 2.2206
Epoch 3/5
15569/15569 [==============================] - 64s 4ms/step - loss: 7.1116 - mean_squared_error: 0.5079 - val_loss: 10.5117 - val_mean_squared_error: 6.4586
Epoch 4/5
15569/15569 [==============================] - 68s 4ms/step - loss: 3.1950 - mean_squared_error: 0.4347 - val_loss: 11.9302 - val_mean_squared_error: 10.4868
Epoch 5/5
15569/15569 [==============================] - 69s 4ms/step - loss: 2.2222 - mean_squared_error: 0.5211 - val_loss: 1.9547 - val_mean_squared_error: 1.2275
Train on 15569 samples, validate on 3893 samples
Epoch 1/5
15569/15569 [==============================] - 70s 4ms/

15571/15571 [==============================] - 70s 5ms/step - loss: 0.0284 - mean_squared_error: 0.0273 - val_loss: 0.0734 - val_mean_squared_error: 0.0723
Epoch 3/5
15571/15571 [==============================] - 67s 4ms/step - loss: 0.0283 - mean_squared_error: 0.0271 - val_loss: 0.0536 - val_mean_squared_error: 0.0524
Epoch 4/5
15571/15571 [==============================] - 62s 4ms/step - loss: 0.0281 - mean_squared_error: 0.0270 - val_loss: 0.0576 - val_mean_squared_error: 0.0564
Epoch 5/5
15571/15571 [==============================] - 64s 4ms/step - loss: 0.0280 - mean_squared_error: 0.0269 - val_loss: 0.0548 - val_mean_squared_error: 0.0536
Train on 15571 samples, validate on 3893 samples
Epoch 1/5
15571/15571 [==============================] - 70s 5ms/step - loss: 0.0274 - mean_squared_error: 0.0263 - val_loss: 0.0536 - val_mean_squared_error: 0.0523
Epoch 2/5
15571/15571 [==============================] - 67s 4ms/step - loss: 0.0280 - mean_squared_error: 0.0268 - val_loss: 0.053

KeyboardInterrupt: 

In [13]:
cvscores

[[0.050954723322117909, 0.049684857715492768],
 [0.036280519784243903, 0.035157474854324591],
 [0.05094815107440958, 0.049548553396647828]]

In [ ]:
i = 0
for train, test in kfold.split(stores, origin_y):
    if i <= 2:
        i += 1
        continue
    model10_stores, instore = ResNet50(input_shape=stores.shape[1:])
    model10_roads, inroad = ResNet50(input_shape=roads.shape[1:])
    model10_pois, in3poi = ResNet50(input_shape=pois.shape[1:])
    
    model10_combine = Three_ResNet(model10_stores, model10_roads, model10_pois, [instore, inroad, in3poi])
    model10_combine.compile(optimizer='adam', loss='mse', metrics=['mse'])
    for _ in range(10):
        lines10_combine = model10_combine.fit([stores[train], roads[train], pois[train]], y_data[train], epochs = 5, batch_size = 16, validation_split=0.2)
        preds10_combine_ = model10_combine.evaluate([stores[test], roads[test], pois[test]], y_data[test])
        print ("model10_combine Loss = " + str(preds10_combine_[0]))
        print ("model10_combine Test Accuracy = " + str(preds10_combine_[1]))
        
    preds10_combine = model10_combine.evaluate([stores[test], roads[test], pois[test]], y_data[test])
    print ("model10_combine Loss = " + str(preds10_combine[0]))
    print ("model10_combine Test Accuracy = " + str(preds10_combine[1]))
    print(preds10_combine)
    cvscores.append(preds10_combine)

Train on 15572 samples, validate on 3893 samples
Epoch 1/5
15572/15572 [==============================] - 71s 5ms/step - loss: 26.2505 - mean_squared_error: 3.4225 - val_loss: 23.3164 - val_mean_squared_error: 5.3536
Epoch 2/5
15572/15572 [==============================] - 67s 4ms/step - loss: 13.7498 - mean_squared_error: 0.5850 - val_loss: 10.0140 - val_mean_squared_error: 1.1401
Epoch 3/5
15572/15572 [==============================] - 68s 4ms/step - loss: 6.2783 - mean_squared_error: 0.3857 - val_loss: 3.8856 - val_mean_squared_error: 0.3751
Epoch 4/5
15572/15572 [==============================] - 65s 4ms/step - loss: 2.4181 - mean_squared_error: 0.2851 - val_loss: 1.3920 - val_mean_squared_error: 0.2742
Epoch 5/5
4866/4866 [==============================] - 4s 826us/step
model10_combine Loss = 0.579778457736
model10_combine Test Accuracy = 0.242134913196
Train on 15572 samples, validate on 3893 samples
Epoch 1/5
15572/15572 [==============================] - 62s 4ms/step - loss: 0.

Train on 15572 samples, validate on 3893 samples
Epoch 1/5
15572/15572 [==============================] - 62s 4ms/step - loss: 0.0284 - mean_squared_error: 0.0261 - val_loss: 0.0712 - val_mean_squared_error: 0.0692
Epoch 2/5
15572/15572 [==============================] - 65s 4ms/step - loss: 0.0289 - mean_squared_error: 0.0261 - val_loss: 0.0715 - val_mean_squared_error: 0.0690
Epoch 3/5
15572/15572 [==============================] - 64s 4ms/step - loss: 0.0272 - mean_squared_error: 0.0248 - val_loss: 0.0803 - val_mean_squared_error: 0.0776
Epoch 4/5
15572/15572 [==============================] - 61s 4ms/step - loss: 0.0282 - mean_squared_error: 0.0254 - val_loss: 0.0678 - val_mean_squared_error: 0.0652
Epoch 5/5
4866/4866 [==============================] - 4s 768us/step
model10_combine Loss = 0.0411698441421
model10_combine Test Accuracy = 0.038723930715
Train on 15572 samples, validate on 3893 samples
Epoch 1/5
15572/15572 [==============================] - 64s 4ms/step - loss: 0.026

Train on 15575 samples, validate on 3894 samples
Epoch 1/5
15575/15575 [==============================] - 69s 4ms/step - loss: 0.0334 - mean_squared_error: 0.0306 - val_loss: 0.0649 - val_mean_squared_error: 0.0627
Epoch 2/5
15575/15575 [==============================] - 64s 4ms/step - loss: 0.0338 - mean_squared_error: 0.0314 - val_loss: 0.0601 - val_mean_squared_error: 0.0578
Epoch 3/5
15575/15575 [==============================] - 68s 4ms/step - loss: 0.0313 - mean_squared_error: 0.0292 - val_loss: 0.0624 - val_mean_squared_error: 0.0606
Epoch 4/5
15575/15575 [==============================] - 69s 4ms/step - loss: 0.0312 - mean_squared_error: 0.0293 - val_loss: 0.0636 - val_mean_squared_error: 0.0619
Epoch 5/5
4862/4862 [==============================] - 4s 773us/step
model10_combine Loss = 0.0417463509285
model10_combine Test Accuracy = 0.0400668136402
Train on 15575 samples, validate on 3894 samples
Epoch 1/5
15575/15575 [==============================] - 71s 5ms/step - loss: 0.03

In [37]:
cvrmse = np.mean(np.sqrt(np.asarray(cvscores)[:, 1]))
cvrmse

0.22232312577059199

In [38]:
cvrmse_stores = []
for train, test in kfold.split(stores, origin_y):
    model10_stores, instore = ResNet50(input_shape=stores.shape[1:])
    model10_roads, inroad = ResNet50(input_shape=stores.shape[1:])
    model10_pois, in3poi = ResNet50(input_shape=stores.shape[1:])
    
    model10_combine = Three_ResNet(model10_stores, model10_roads, model10_pois, [instore, inroad, in3poi])
    model10_combine.compile(optimizer='adam', loss='mse', metrics=['mse'])
    lines10_combine = model10_combine.fit([stores[train], stores[train], stores[train]], y_data[train], epochs = 50, batch_size = 16, validation_split=0.2)

    preds10_combine = model10_combine.evaluate([stores[test], stores[test], stores[test]], y_data[test])
    print ("model10_combine Loss = " + str(preds10_combine[0]))
    print ("model10_combine Test Accuracy = " + str(preds10_combine[1]))
    print(preds10_combine)
    cvrmse_stores.append(preds10_combine)

Train on 15569 samples, validate on 3893 samples
Epoch 1/50
15569/15569 [==============================] - 93s 6ms/step - loss: 22.5843 - mean_squared_error: 1.4465 - val_loss: 16.5381 - val_mean_squared_error: 1.7199
Epoch 2/50
15569/15569 [==============================] - 73s 5ms/step - loss: 10.4134 - mean_squared_error: 0.4796 - val_loss: 6.2208 - val_mean_squared_error: 0.3534
Epoch 3/50
15569/15569 [==============================] - 71s 5ms/step - loss: 3.9632 - mean_squared_error: 0.3604 - val_loss: 2.1548 - val_mean_squared_error: 0.2657
Epoch 4/50
15569/15569 [==============================] - 72s 5ms/step - loss: 1.2880 - mean_squared_error: 0.2198 - val_loss: 0.8597 - val_mean_squared_error: 0.3460
Epoch 5/50
15569/15569 [==============================] - 72s 5ms/step - loss: 0.4901 - mean_squared_error: 0.1805 - val_loss: 0.3894 - val_mean_squared_error: 0.2387
Epoch 6/50
15569/15569 [==============================] - 71s 5ms/step - loss: 0.2487 - mean_squared_error: 0.142

15571/15571 [==============================] - 71s 5ms/step - loss: 0.0289 - mean_squared_error: 0.0267 - val_loss: 0.0737 - val_mean_squared_error: 0.0718
Epoch 47/50
15571/15571 [==============================] - 71s 5ms/step - loss: 0.0292 - mean_squared_error: 0.0269 - val_loss: 0.0701 - val_mean_squared_error: 0.0684
Epoch 48/50
15571/15571 [==============================] - 71s 5ms/step - loss: 0.0286 - mean_squared_error: 0.0265 - val_loss: 0.0638 - val_mean_squared_error: 0.0619
Epoch 49/50
15571/15571 [==============================] - 72s 5ms/step - loss: 0.0285 - mean_squared_error: 0.0264 - val_loss: 0.0668 - val_mean_squared_error: 0.0637
Epoch 50/50
4867/4867 [==============================] - 7s 1ms/step
model10_combine Loss = 0.0467052270536
model10_combine Test Accuracy = 0.0448326347777
[0.046705227053603866, 0.044832634777657324]
Train on 15571 samples, validate on 3893 samples
Epoch 1/50
15571/15571 [==============================] - 93s 6ms/step - loss: 22.6284 - m

15572/15572 [==============================] - 71s 5ms/step - loss: 0.0295 - mean_squared_error: 0.0264 - val_loss: 0.0828 - val_mean_squared_error: 0.0805
Epoch 41/50
15572/15572 [==============================] - 71s 5ms/step - loss: 0.0295 - mean_squared_error: 0.0267 - val_loss: 0.0823 - val_mean_squared_error: 0.0800
Epoch 42/50
15572/15572 [==============================] - 71s 5ms/step - loss: 0.0285 - mean_squared_error: 0.0259 - val_loss: 0.0770 - val_mean_squared_error: 0.0745
Epoch 43/50
15572/15572 [==============================] - 71s 5ms/step - loss: 0.0276 - mean_squared_error: 0.0249 - val_loss: 0.0794 - val_mean_squared_error: 0.0773
Epoch 44/50
15572/15572 [==============================] - 71s 5ms/step - loss: 0.0272 - mean_squared_error: 0.0249 - val_loss: 0.0817 - val_mean_squared_error: 0.0790
Epoch 45/50
15572/15572 [==============================] - 71s 5ms/step - loss: 0.0275 - mean_squared_error: 0.0250 - val_loss: 0.0850 - val_mean_squared_error: 0.0823
Epoc

In [40]:
cvscores = [[0.050737945057266573, 0.048019854719510535],
 [0.047062903559989253, 0.044174419010667702],
 [0.058294292038009232, 0.054615884224032879],
 [0.056042961626203765, 0.05218257700379033],
 [0.051719239743978879, 0.048474260321226398]]

In [42]:
np.sqrt(np.mean(np.asarray(cvscores)[:,1]))

0.22247111959947874

In [43]:
cv2 = [[0.050971167342433954, 0.047386268105106932], [0.046705227053603866, 0.044832634777657324], [0.054852741401323381, 0.053387577191396293], [0.053468271367254933, 0.051538033399423158], [0.049991687336045497, 0.04763181455102368]]
np.sqrt(np.mean(np.asarray(cv2)[:,1]))

0.2212583684404309